## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
0,0,Kinna,57.5073,12.6946,55.35,92,85,8.30,SE,overcast clouds,2021-10-10 23:19:58
1,1,Cape Town,-33.9258,18.4232,63.12,85,97,5.35,ZA,overcast clouds,2021-10-10 23:13:50
2,2,Carnarvon,-24.8667,113.6333,66.43,68,0,21.25,AU,clear sky,2021-10-10 23:12:02
3,3,Rikitea,-23.1203,-134.9692,74.80,80,98,18.25,PF,overcast clouds,2021-10-10 23:10:31
4,4,Hithadhoo,-0.6000,73.0833,82.62,68,99,14.43,MV,overcast clouds,2021-10-10 23:14:06


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature would you prefer for your trip?'))
max_temp = float(input('What is the maximum temperation would you prefer for your trip?'))

What is the minimum temperature would you prefer for your trip?75
What is the maximum temperation would you prefer for your trip?95


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_city_df = city_data_df.loc[(city_data_df['Max Temp']>=min_temp) & \
                                      (city_data_df['Max Temp']<=max_temp)]
preferred_city_df.head(10)

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
4,4,Hithadhoo,-0.6000,73.0833,82.62,68,99,14.43,MV,overcast clouds,2021-10-10 23:14:06
6,6,Can Tho,10.0333,105.7833,78.78,89,40,2.30,VN,scattered clouds,2021-10-10 23:19:59
7,7,Karaton,-6.3424,106.1045,81.95,88,72,0.96,ID,broken clouds,2021-10-10 23:19:59
10,10,Kapaa,22.0752,-159.3190,84.18,71,1,5.99,US,light rain,2021-10-10 23:13:43
13,13,Tshikapa,-6.4167,20.8000,76.21,68,72,3.53,CD,broken clouds,2021-10-10 23:20:00
18,18,San Patricio,28.0170,-97.5169,91.49,64,46,20.27,US,scattered clouds,2021-10-10 23:14:12
20,20,Avarua,-21.2078,-159.7750,77.05,73,100,10.36,CK,overcast clouds,2021-10-10 23:14:09
25,25,Bara,10.3744,10.7288,76.35,76,95,7.47,NG,overcast clouds,2021-10-10 23:20:02
26,26,Lompoc,34.6391,-120.4579,81.09,62,1,11.50,US,clear sky,2021-10-10 23:12:09
28,28,Praia,14.9215,-23.5087,79.34,78,20,16.11,CV,few clouds,2021-10-10 23:12:18


In [6]:
# 4a. Determine if there are any empty rows.
preferred_city_df.count()

City ID                230
City                   230
Lat                    230
Lng                    230
Max Temp               230
Humidity               230
Cloudiness             230
Wind Speed             230
Country                230
Current Description    230
Date                   230
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_city_df.dropna()
clean_df.count()

City ID                230
City                   230
Lat                    230
Lng                    230
Max Temp               230
Humidity               230
Cloudiness             230
Wind Speed             230
Country                230
Current Description    230
Date                   230
dtype: int64

In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Hithadhoo,MV,82.62,overcast clouds,-0.6000,73.0833,
6,Can Tho,VN,78.78,scattered clouds,10.0333,105.7833,
7,Karaton,ID,81.95,broken clouds,-6.3424,106.1045,
10,Kapaa,US,84.18,light rain,22.0752,-159.3190,
13,Tshikapa,CD,76.21,broken clouds,-6.4167,20.8000,
18,San Patricio,US,91.49,scattered clouds,28.0170,-97.5169,
20,Avarua,CK,77.05,overcast clouds,-21.2078,-159.7750,
25,Bara,NG,76.35,overcast clouds,10.3744,10.7288,
26,Lompoc,US,81.09,clear sky,34.6391,-120.4579,
28,Praia,CV,79.34,few clouds,14.9215,-23.5087,


In [26]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")    
        

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [28]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df.loc[:,"Hotel Name"] != "")]

In [29]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [33]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [32]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4) 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))